In [1]:
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import time     # this library provides various time related functions like current time, date etc.

In [2]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Read the initial data for different states
data_CA=pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/California/Data_California_copy.csv")#California
data_WA=pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/Washington/Data_Washington_copy.csv")#Washington
data_OR=pd.read_csv("/content/drive/My Drive/AI_Thermal sytems_course_project/Oregon/Data_Oregon_copy.csv")#Oregon
dc=[data_WA, data_OR, data_CA]
data_tot=pd.concat(dc,ignore_index=True)# Merge datas of different states
data_tot.sort_values('County',inplace=True)# Sort alphabetically
data_tot.reset_index(inplace=True,drop=True)# reset the indexing
from pandas.api.types import CategoricalDtype # import module required to make a categorical data
status_type = CategoricalDtype(categories=['increase', 'decrease','no change'], ordered=True) #define your items as categories
data_tot['status of the change'] = data_tot['status of the change'].astype(status_type)
data_tot_copy = data_tot.copy(deep=True)
data_CA.dtypes

County                                            object
Population                                       float64
Area (sq km)                                     float64
Population density (person/km^2)                 float64
High school percentage                           float64
Persons per household                            float64
Persons in poverty percentage                    float64
Median income                                    float64
Biden won                                          int64
Trump won                                          int64
Never                                            float64
Rarely                                           float64
Sometimes                                        float64
Frequently                                       float64
Always                                           float64
1_months_before_order deaths/cases percentage    float64
1_months_after_order deaths/cases percentage     float64
change in deaths/cases percenta

In [4]:
data_tot.dtypes

County                                             object
Population                                        float64
Area (sq km)                                      float64
Population density (person/km^2)                  float64
High school percentage                            float64
Persons per household                             float64
Persons in poverty percentage                     float64
Median income                                     float64
Biden won                                           int64
Trump won                                           int64
Never                                             float64
Rarely                                            float64
Sometimes                                         float64
Frequently                                        float64
Always                                            float64
1_months_before_order deaths/cases percentage     float64
1_months_after_order deaths/cases percentage      float64
change in deat

In [5]:
data_tot.describe()

,Population,Area (sq km),Population density (person/km^2),High school percentage,Persons per household,Persons in poverty percentage,Median income,Biden won,Trump won,Never,Rarely,Sometimes,Frequently,Always,1_months_before_order deaths/cases percentage,1_months_after_order deaths/cases percentage,change in deaths/cases percentage
count,1.300000e+02,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000
mean,3.903071e+05,6275.789030,150.642998,87.054615,2.608308,12.916923,61403.500000,0.423077,0.576923,0.035331,0.038262,0.071508,0.174492,0.680292,0.994753,0.715021,-0.279733
std,1.037801e+06,6525.471194,664.180979,6.576740,0.363344,3.619438,16712.599359,0.495959,0.495959,0.028952,0.034571,0.037053,0.054979,0.096594,1.840788,1.227060,2.184181
min,1.129000e+03,121.392831,0.274595,66.900000,2.030000,6.100000,37898.000000,0.000000,0.000000,0.001000,0.000000,0.004000,0.058000,0.305000,0.000000,0.000000,-12.903226
25%,2.446750e+04,2487.452296,5.166023,84.575000,2.342500,10.600000,51109.250000,0.000000,0.000000,0.016000,0.014000,0.048000,0.141500,0.615250,0.000000,0.000000,-0.775084
50%,7.830900e+04,4462.552770,18.625742,89.250000,2.515000,12.650000,55952.000000,0.000000,1.000000,0.026000,0.028000,0.069000,0.168500,0.679500,0.000000,0.343842,0.000000
75%,2.817532e+05,7698.557501,61.052002,91.475000,2.810000,15.175000,67022.250000,1.000000,1.000000,0.045000,0.056750,0.090500,0.204000,0.753750,1.586655,1.010384,0.000000
max,1.003911e+07,51947.274030,7261.952708,95.900000,3.870000,26.300000,124055.000000,1.000000,1.000000,0.140000,0.206000,0.213000,0.332000,0.889000,12.903226,7.692308,7.692308


In [9]:
!pip install --upgrade plotly
!pip install --upgrade plotly-geo
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)
Requirement already up-to-date: plotly-geo in /usr/local/lib/python3.7/dist-packages (1.0.0)
Requirement already up-to-date: geopandas in /usr/local/lib/python3.7/dist-packages (0.8.2)
Requirement already up-to-date: pyshp in /usr/local/lib/python3.7/dist-packages (2.1.3)
Requirement already up-to-date: shapely in /usr/local/lib/python3.7/dist-packages (1.7.1)


In [11]:
import plotly.figure_factory as ff

import pandas as pd

NE_states = ['Washington', 'Oregon', 'California']
df_sample = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/minoritymajority.csv')
df_sample_r = df_sample[df_sample['STNAME'].isin(NE_states)]

df_sample_r.drop(df_sample_r.loc[df_sample_r["FIPS"] == 53029].index, inplace=True)
df_sample_r.drop(df_sample_r.loc[df_sample_r["FIPS"] == 53067].index, inplace=True)
df_sample_r.drop(df_sample_r.loc[df_sample_r["FIPS"] == 53073].index, inplace=True)

values = data_tot['change in deaths/cases percentage']
fips = df_sample_r['FIPS'].tolist()

colorscale = [
    'rgb(68.0, 1.0, 84.0)',
    'rgb(66.0, 64.0, 134.0)',
    'rgb(38.0, 130.0, 142.0)',
    'rgb(63.0, 188.0, 115.0)',
    'rgb(216.0, 226.0, 25.0)'
]

fig = ff.create_choropleth(
    fips=fips, values=values,
    scope=NE_states, 
    binning_endpoints=[data_tot['change in deaths/cases percentage'].quantile(.01), data_tot['change in deaths/cases percentage'].quantile(.25) , data_tot['change in deaths/cases percentage'].median(), data_tot['change in deaths/cases percentage'].quantile(.77), data_tot['change in deaths/cases percentage'].max()],
    county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
    legend_title='Change in death ratio per county (%)', title='USA West Coast States'

)
fig.update_layout()

fig.layout.template = None
fig.show()
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
os.chdir('/content/drive/My Drive/AI_Thermal sytems_course_project/figures')

import io
from PIL import Image

!pip install -U kaleido
import plotly.graph_objects as go

!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
fig.write_image(file='death_ratio_county_map.png', format='.png' , scale = 1.2) 

Requirement already up-to-date: kaleido in /usr/local/lib/python3.7/dist-packages (0.1.0)
/usr/local/bin/orca: Text file busy
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgtk2.0-0 is already the newest version (2.24.32-1ubuntu1).
libgconf-2-4 is already the newest version (3.2.6-4ubuntu1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
